In [1]:
import bokeh.io

bokeh.io.output_notebook()

Loading BokehJS ...

In [17]:
import bokeh.plotting
import bokeh.models

import numpy as np

def plot_solution(solution):
    values = solution.values
    
    len_s = values.shape[0]
    len_t = values.shape[1]
    cell_size = 30

    p = bokeh.plotting.figure(
        plot_width=len_t * cell_size, plot_height=len_s * cell_size,
        title=None, toolbar_location=None)

    len_s_1 = len_s + 1
    len_t_1 = len_t + 1
    grid_line_width = 0.5

    p.multi_line(
        xs=(1 + np.array([[0, len_t]] * len_s_1) - 0.5).tolist(),
        ys=(np.repeat(1 + np.arange(0, len_s_1), 2).reshape(len_s_1, 2) - 0.5).tolist(),
        color="gray", line_width=grid_line_width)

    p.multi_line(
        xs=(np.repeat(1 + np.arange(0, len_t_1), 2).reshape(len_t_1, 2) - 0.5).tolist(),
        ys=(1 + np.array([[0, len_s]] * len_t_1) - 0.5).tolist(),
        color="gray", line_width=grid_line_width)
    
    path_mask = np.logical_and(solution.path[:, 0] >= 0, solution.path[:, 1] >= 0)
    #p.scatter(
    #    solution.path[path_mask, 0] + 1, solution.path[path_mask, 1] + 1,
    #    marker="circle", fill_color="orange", alpha=0.25, radius=0.5)
    
    source = bokeh.models.ColumnDataSource(dict(
        x=solution.path[path_mask, 0] + 1,
        y=solution.path[path_mask, 1] + 1,
        width=[1] * np.sum(path_mask),
        height=[1] * np.sum(path_mask)))
    
    glyph = bokeh.models.Rect(
        x="x", y="y", width="width", height="height",
        fill_color="orange",
        fill_alpha=0.25)
    p.add_glyph(source, glyph)
    
    source = bokeh.models.ColumnDataSource(
        data=dict(
            x=1 + np.tile(np.arange(0, len_t), len_s),
            y=1 + np.repeat(np.arange(0, len_s), len_t),
            value=[f"{x:.1f}" for x in values.flatten()]))

    labels = bokeh.models.LabelSet(
        x='x', y='y', text='value',
        x_offset=0, y_offset=0, source=source, render_mode='canvas',
        text_font_size='9pt', text_align='center', text_baseline='middle')
    
    p.add_layout(labels)

    #if solution.problem.s is not None:
    #p.xaxis.ticker = bokeh.models.FixedTicker(ticks=solution.problem.s)

    p.xaxis.ticker = bokeh.models.FixedTicker(ticks=np.arange(0, len_t) + 1)
    p.yaxis.ticker = bokeh.models.FixedTicker(ticks=np.arange(0, len_s) + 1)
    p.y_range.flipped = True

    p.grid.grid_line_color = None
    
    bokeh.io.show(p)

In [19]:
import pyalign.utils
import pyalign.solve
import pyalign.gaps

pf = pyalign.utils.SimpleProblemFactory(pyalign.utils.BinarySimilarity(eq=1, ne=-1))
solver = pyalign.solve.Solver(gap_cost=pyalign.gaps.AffineGapCost(0))
alignment = solver.solve(pf.new_problem("INDUSTRY", "INTEREST"))
print(alignment.score)
alignment.print()

4.0
[ 0  1 -1 -1  6  7 -1 -1]
INDU    STRY
||      ||  
IN  TEREST  


In [20]:
solver.solution.values

array([[1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 2., 2., 2., 2., 2., 2., 2.],
       [1., 2., 2., 2., 2., 2., 2., 2.],
       [1., 2., 2., 2., 2., 2., 2., 2.],
       [1., 2., 2., 2., 2., 2., 3., 3.],
       [1., 2., 3., 3., 3., 3., 3., 4.],
       [1., 2., 3., 3., 4., 4., 4., 4.],
       [1., 2., 3., 3., 4., 4., 4., 4.]], dtype=float32)

In [9]:
solver.solution.path

array([[ 5,  7],
       [ 4,  6],
       [ 3,  5],
       [ 2,  5],
       [ 1,  5],
       [ 1,  4],
       [ 1,  3],
       [ 1,  2],
       [ 1,  1],
       [ 0,  0],
       [-1, -1]], dtype=int16)

In [21]:
plot_solution(solver.solution)